In [40]:
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import psycopg2  
import subprocess 
import sys, os
import math

import stac
from osgeo import gdal, ogr, osr
import rasterio

from datetime import datetime

import ipywidgets as widgets

## Obtenção dos itens da coleção do BDC

In [63]:
# Conectar o serviço da stac
bdc_stac_service = stac.STAC('http://brazildatacube.dpi.inpe.br/')

# Obter itens filtrados pelo objeto JSON (https://stacspec.org/STAC-api.html#operation/postSearchSTAC)
lc8_root = bdc_stac_service.search({'collections':['LC8SR-1'], 
                                    'limit': 100000})

lc8_bdc = bdc_stac_service.search({'collections':['LC8_30_16D_STK-1'], 
                                    'limit': 100000})

s2_root = bdc_stac_service.search({'collections':['S2_MSI_L2_SR_LASRC-1'], 
                                    'limit': 100000})

s2_bdc = bdc_stac_service.search({'collections':['S2_10_16D_STK-1'], 
                                    'limit': 100000})

list_collections = [lc8_root, lc8_bdc, s2_root, s2_bdc]

# Exibir as quantidades de itens filtrados
print("LC8SR-1 - ", len(lc8_root.features))
print("LC8_30_16D_STK-1 - ", len(lc8_bdc.features))
print("S2_MSI_L2_SR_LASRC-1 - ", len(s2_root.features))
print("S2_10_16D_STK-1 - ", len(s2_bdc.features))

LC8SR-1 -  33239
LC8_30_16D_STK-1 -  3381
S2_MSI_L2_SR_LASRC-1 -  24791
S2_10_16D_STK-1 -  852


In [71]:
s2_root.features[0]['properties']['bdc:tile']

'21LXC'

## Conexão com o banco de dados 'bdc3'

In [64]:
conn = psycopg2.connect(database="bdc3", user="postgres", host="localhost", password="postgres") 
cursor = conn.cursor()

### Criar tabela com os metadados das imagens a serem inseridas

Essa tabela já pode ser inserida no momento da criação do banco. Nesse caso, a célula seguinte seria desconsiderada.

**<center>Colunas da tabela</center>**

| id        | collection           | date_img  | cover |
| :-------------:|:-------------:|:-----:|:-----:|
| VARCHAR(60) NOT NULL UNIQUE     | VARCHAR(30) | DATE | REAL |

In [73]:
# Criar tabela
sql = "CREATE TABLE metadata_metrics(id VARCHAR(60) NOT NULL UNIQUE, pathrow VARCHAR(6), collection VARCHAR(60), date_img DATE, cover REAL);"
cursor.execute(sql)
conn.commit()

### Importação das imagens filtradas para o BD (cmd)

In [74]:
# Criação das variáveis do sistema para utilização do CMD
os.environ['PATH'] = r';C:\Program Files\PostgreSQL\11\bin'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'
os.environ['PGUSER'] = 'postgres'
os.environ['PGPASSWORD'] = 'postgres'
os.environ['PGDATABASE'] = 'bdc3'

# iteração em cada item adquirido para inserção das imagens no DB "bdc3"
for x in list_collections:
    
    print(x.features[0]['collection'])
    
    for i in x.features:
    
        # Adquirir os dados das imagens
        name = i['id']
        date = i['properties']['datetime']
        collection = i['collection']
        cloud = str(i['properties']['eo:cloud_cover'])
        path_row = i['properties']['bdc:tile']


        if cloud == 'None':
            print(name)
            cloud = '0'
        
        # Inserir a referencia do input na tabela de metadados
        sql = "INSERT INTO metadata_metrics (id, collection, pathrow, date_img, cover) VALUES ('" + name + "','" + collection + "','" + path_row + "','" + date + "'::date, "+ cloud + ");"
        cursor.execute(sql)
        conn.commit()

LC8SR-1
LC08_L1TP_224078_20191226_20200110_01_T1
LC08_L1TP_224078_20191210_20191217_01_T1
LC08_L1TP_224078_20191124_20191203_01_T1
LC08_L1GT_002067_20191121_20191203_01_T2
LC08_L1TP_224078_20191108_20191115_01_T1
LC08_L1TP_224078_20191023_20191030_01_T1
LC08_L1TP_224078_20191007_20191018_01_T1
LC08_L1TP_224078_20190921_20190926_01_T1
LC08_L1TP_224078_20190905_20190917_01_T1
LC08_L1TP_224078_20190820_20190903_01_T1
LC08_L1TP_224078_20190804_20190820_01_T1
LC08_L1GT_224078_20190719_20190731_01_T2
LC08_L1TP_224078_20190703_20190718_01_T1
LC08_L1TP_224078_20190617_20190620_01_T1
LC08_L1GT_224078_20190601_20190605_01_T2
LC08_L1TP_224078_20190516_20190521_01_T1
LC08_L1TP_224078_20190430_20190508_01_T1
LC08_L1TP_224078_20190414_20190422_01_T1
LC08_L1TP_224078_20190329_20190404_01_T1
LC08_L1GT_224078_20190313_20190325_01_T2
LC08_L1TP_224078_20190225_20190309_01_T1
LC08_L1TP_224078_20190209_20190222_01_T1
LC08_L1TP_224078_20190124_20190205_01_T1
LC08_L1GT_224078_20190108_20190130_01_T2
LC08_L1T

### Apagar tabela (Interno)

In [72]:
sql = "DROP TABLE IF EXISTS metadata_metrics"
cursor.execute(sql)
conn.commit()